In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.__version__

'2.0.0'

# 당뇨병을 진단해봅시다.

In [2]:
xy = np.loadtxt('../Day_1/data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_train = xy[0:-100, 0:-1]
y_train = xy[0:-100, [-1]]

x_test = xy[-100:, 0:-1]
y_test = xy[-100:, [-1]]

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(659, 8) (659, 1)
(100, 8) (100, 1)


In [3]:
print(x_test)
print(y_test)

[[-0.294118   -0.0150754  -0.0491803  -0.333333   -0.550827    0.0134128
  -0.699402   -0.266667  ]
 [ 0.0588235   0.547739    0.278689   -0.393939   -0.763593   -0.0789866
  -0.926558   -0.2       ]
 [-0.294118    0.658291    0.114754   -0.474747   -0.602837    0.00149028
  -0.527754   -0.0666667 ]
 [-0.882353   -0.00502513 -0.0491803  -0.79798     0.         -0.242921
  -0.596072    0.        ]
 [ 0.176471   -0.316583    0.737705   -0.535354   -0.884161    0.0581222
  -0.823228   -0.133333  ]
 [-0.647059    0.236181    0.639344   -0.292929   -0.432624    0.707899
  -0.315115   -0.966667  ]
 [-0.0588235  -0.0854271   0.344262    0.          0.          0.0611028
  -0.565329    0.566667  ]
 [-0.294118    0.959799    0.147541    0.          0.         -0.0789866
  -0.786507   -0.666667  ]
 [ 0.0588235   0.567839    0.409836    0.          0.         -0.260805
  -0.870196    0.0666667 ]
 [ 0.         -0.0653266  -0.0163934   0.          0.          0.052161
  -0.842015   -0.866667  ]
 [-

In [4]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))


In [5]:
W = tf.Variable(tf.random.normal([8, 1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')


## 가설 설정
* 병이 있다 / 없다로 분류
* softmax classification으로 진행

## $$ y_k = \frac{exp(x_k)}{\sum_{i=1}^{n}exp(x_i)}  $$

In [7]:
def logistic_regression(features):
    hypothesis  = tf.divide(1., 1. + tf.exp(-(tf.matmul(features, W) + b)))
    return hypothesis
  
print(logistic_regression(x_train))

tf.Tensor(
[[0.804652  ]
 [0.64367676]
 [0.9335452 ]
 [0.6629444 ]
 [0.93606865]
 [0.8441041 ]
 [0.81897813]
 [0.8937661 ]
 [0.872246  ]
 [0.79233646]
 [0.7896296 ]
 [0.91843855]
 [0.87188834]
 [0.6620156 ]
 [0.8670559 ]
 [0.8739122 ]
 [0.8440871 ]
 [0.90648437]
 [0.79305893]
 [0.82069695]
 [0.79610157]
 [0.90374666]
 [0.7690115 ]
 [0.81018347]
 [0.87906826]
 [0.6478054 ]
 [0.7513306 ]
 [0.79664296]
 [0.62701184]
 [0.8587991 ]
 [0.6634346 ]
 [0.761917  ]
 [0.73935604]
 [0.8174561 ]
 [0.89028484]
 [0.7545599 ]
 [0.7778311 ]
 [0.86100775]
 [0.8800579 ]
 [0.8588924 ]
 [0.48495826]
 [0.7117575 ]
 [0.90809095]
 [0.9203857 ]
 [0.9022299 ]
 [0.60910165]
 [0.75882685]
 [0.8785529 ]
 [0.579625  ]
 [0.74187374]
 [0.7296619 ]
 [0.8148696 ]
 [0.88543904]
 [0.45652282]
 [0.905727  ]
 [0.9188971 ]
 [0.8899228 ]
 [0.8870539 ]
 [0.80117315]
 [0.8767839 ]
 [0.7706921 ]
 [0.8807777 ]
 [0.8615512 ]
 [0.65225804]
 [0.6812815 ]
 [0.7601266 ]
 [0.6447635 ]
 [0.7904729 ]
 [0.726951  ]
 [0.87738335]
 [0.85890

## Loss Function

* 기존 MSE 대신 Cross Entropy 사용

## $$
\begin{align}
loss(h(x),y) & = −y log(h(x))−(1−y)log(1−h(x))
\end{align}
$$

In [11]:
def loss_fn(hypothesis, labels):
    loss = -tf.reduce_mean(labels * tf.math.log(hypothesis) + \
                           (1 - labels) * tf.math.log(1 - hypothesis))
    return loss

optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.005)

## 학습

In [12]:
epochs = 5000

for step in range(epochs):
  for features, labels in dataset:
    with tf.GradientTape() as tape:
      loss_value = loss_fn(logistic_regression(features),labels)
      grads = tape.gradient(loss_value, [W,b])
      optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
      if step % 100 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, loss_fn(logistic_regression(features),labels)))

Iter: 0, Loss: 0.8272
Iter: 100, Loss: 0.8098
Iter: 200, Loss: 0.7955
Iter: 300, Loss: 0.7834
Iter: 400, Loss: 0.7730
Iter: 500, Loss: 0.7637
Iter: 600, Loss: 0.7553
Iter: 700, Loss: 0.7476
Iter: 800, Loss: 0.7403
Iter: 900, Loss: 0.7334
Iter: 1000, Loss: 0.7268
Iter: 1100, Loss: 0.7205
Iter: 1200, Loss: 0.7144
Iter: 1300, Loss: 0.7086
Iter: 1400, Loss: 0.7029
Iter: 1500, Loss: 0.6974
Iter: 1600, Loss: 0.6921
Iter: 1700, Loss: 0.6869
Iter: 1800, Loss: 0.6819
Iter: 1900, Loss: 0.6770
Iter: 2000, Loss: 0.6723
Iter: 2100, Loss: 0.6677
Iter: 2200, Loss: 0.6632
Iter: 2300, Loss: 0.6589
Iter: 2400, Loss: 0.6547
Iter: 2500, Loss: 0.6506
Iter: 2600, Loss: 0.6467
Iter: 2700, Loss: 0.6428
Iter: 2800, Loss: 0.6391
Iter: 2900, Loss: 0.6355
Iter: 3000, Loss: 0.6319
Iter: 3100, Loss: 0.6285
Iter: 3200, Loss: 0.6252
Iter: 3300, Loss: 0.6220
Iter: 3400, Loss: 0.6188
Iter: 3500, Loss: 0.6158
Iter: 3600, Loss: 0.6128
Iter: 3700, Loss: 0.6099
Iter: 3800, Loss: 0.6071
Iter: 3900, Loss: 0.6044
Iter: 4000, 

## 테스트

In [13]:
def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.int32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy

In [14]:
test_acc = accuracy_fn(logistic_regression(x_test),y_test)
print("Testset Accuracy: {:.4f}".format(test_acc))

Testset Accuracy: 0.6700
